In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import random
import tqdm
import pyautogui
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By
import datetime

In [2]:
browser = webdriver.Chrome()

In [3]:
url = 'https://yandex.ru/maps/?display-text=%D0%BA%D0%B0%D0%BB%D1%83%D0%B6%D1%81%D0%BA%D0%B0%D1%8F%20%D0%BE%D0%B1%D0%BB%D0%B0%D1%81%D1%82%D1%8C%20%D0%B1%D0%B0%D0%BD%D0%BA%D0%BE%D0%BC%D0%B0%D1%82%D1%8B%20%D1%81%D0%B1%D0%B5%D1%80%D0%B1%D0%B0%D0%BD%D0%BA&ll=35.715894%2C54.640877&mode=search&sll=38.204526%2C55.019035&sspn=10.058739%2C4.136658&text=%D0%BA%D0%B0%D0%BB%D1%83%D0%B6%D1%81%D0%BA%D0%B0%D1%8F%20%D0%BE%D0%B1%D0%BB%D0%B0%D1%81%D1%82%D1%8C%20%D0%B1%D0%B0%D0%BD%D0%BA%D0%BE%D0%BC%D0%B0%D1%82%D1%8B%20%D1%81%D0%B1%D0%B5%D1%80%D0%B1%D0%B0%D0%BD%D0%BA&z=7.27'
browser.get(url)

In [4]:
inp = browser.find_element(By.XPATH,'/html/body/div[1]/div[2]/div[3]/header/div/div/div/form/div[2]/div/span/span/input')

In [5]:
inp.send_keys(Keys.CONTROL + "a")
inp.send_keys(Keys.DELETE)
inp.send_keys("Камчатский край банкоматы Сбербанк")
time.sleep(3)
inp.send_keys(Keys.ENTER)
time.sleep(5)

In [6]:
h = ''
elem = browser.find_elements(By.CLASS_NAME,'search-business-snippet-view')
while h != elem[-1]:
    h = elem[-1]
    browser.execute_script("arguments[0].scrollIntoView(true);",elem[-1])
    time.sleep(5)
    elem = browser.find_elements(By.CLASS_NAME,'search-business-snippet-view')
    
    time.sleep(2)

In [7]:
ee = browser.find_elements(By.XPATH,'//a[@class="search-snippet-view__link-overlay _focusable"]')
Links = [ee[i].get_attribute('href') for i in range(len(ee))]

In [14]:
df = pd.DataFrame(columns = ['link', 'adr','work','ost','otz','osob','inf', 'time_added'])

In [ ]:
for j in Links:
    browser.get(j)
    time.sleep(5)
    browser.find_elements(By.CLASS_NAME, 'business-working-status-view')[0].click()
    time.sleep(2)
    work = ''
    el_d = browser.find_elements(By.CLASS_NAME, 'business-working-intervals-view__day')
    el_t = browser.find_elements(By.CLASS_NAME, 'business-working-intervals-view__intervals')
    if len(el_d) != len(el_t):
        print('error')
        continue
    else:
        for i in range(len(el_d)):
            work = work + el_d[i].text + ' : '
            work = work + el_t[i].text + '/n'

    close = browser.find_elements(By.CLASS_NAME, 'close-button')
    if len(close) > 0:
        close[0].click()
    time.sleep(1)

    link = j

    adr = browser.find_elements(By.CLASS_NAME, 'business-contacts-view__address-link')
    if len(adr) > 0:
        adr = adr[0].text
    else:
        adr = ''

    ost = browser.find_element(By.CLASS_NAME, 'masstransit-stops-view__stop-name')
    metrs = browser.find_element(By.CLASS_NAME, 'masstransit-stops-view__stop-distance')
    ost = ost.text + '  ' + metrs.text

    features = browser.find_elements(By.CLASS_NAME, 'business-features-view__bool-text')

    vkl = browser.find_elements(By.CLASS_NAME, 'tabs-select-view__title')

    vkl[2].click()
    time.sleep(5)

    o = browser.find_elements(By.CLASS_NAME, 'business-review-view__body-text')
    otz = []
    for i in range(len(o)):
        otz.append(o[i].text)

    vkl[4].click()
    time.sleep(5)

    o = browser.find_elements(By.CLASS_NAME, 'business-features-view__bool-text')
    osob = []
    for i in range(len(o)):
        osob.append(o[i].text)

    time.sleep(3)

    o = browser.find_elements(By.CLASS_NAME, 'business-features-view__valued-value')
    oper = []
    for i in range(len(o)):
        oper.append(o[i].text)

    current_time = datetime.datetime.now()

    df = df.append({'link': link, 'adr': adr, 'work': work, 'ost': ost, 'otz': otz,
                    'osob': osob, 'inf': oper, 'time_added': current_time}, ignore_index=True)

In [16]:
df.to_excel('Камчатский.xlsx')